# SADS 모델의 Unicast Phase 동작 검증

* 공격자의 광고 패킷이 존재하지 않는다는 가정 하에 동작 검증
* 사용자의 광고 패킷을 스푸핑 공격으로 감지하는 FN (= False Negative) 검증

In [8]:
import pandas as pd
import random

In [9]:
# seed = 25632
random.seed("25632")

In [10]:
source_file = './data/false-negative/data1_fn.csv'

df = pd.read_csv(source_file)
df_time = df['time']

In [11]:
print("최초 입력 시간 :", df_time[0])
print("마지막 입력 시간 :", df_time[len(df_time)-1])
print("데이터 갯수 :",len(df_time))
print("총 소요 시간 :", df_time[len(df_time)-1] - df_time[0])

최초 입력 시간 : 1641964485.5144866
마지막 입력 시간 : 1641964899.6425989
데이터 갯수 : 2000
총 소요 시간 : 414.1281123161316


In [12]:
end_time = df_time[len(df_time)-1]
curr_time = df_time[0]

time_list = []
time_list.append(curr_time)

idx = 0

# Time List's Odd Index => Promise Time's End
# Time List's Even Index => Non-promise Time's End
while (curr_time <= end_time):
    
    if idx % 2 == 0:
        # Promise Time에서 Non-promise Time으로 넘어갈 때의 False Alarm 방지를 위한 유예 시간 (= 0.5)
        v = random.randint(5, 7) + 0.5
        time_list.append(curr_time + v)
        curr_time += v
    else:
        # Non-promise Time에서 Promise Time으로 넘어갈 때의 False Alarm 방지를 위한 유예 시간 (= 0.5)
        v = random.randint(2, 5) - 1
        time_list.append(curr_time + v)
        time_list.append(curr_time + v + 0.5)
        curr_time = curr_time + v + 0.5
        
    idx += 1
    
print("Time List's Length :", len(time_list))
# print("Time List :", time_list)

Time List's Length : 131


In [13]:
# False Negative
# Non-promise Time일때 정상 Tag 감지
np_true_cnt = 0

idx = 0

for i in range(df.shape[0]):
    tmp = idx
    
    for j in range(idx, len(time_list)):
        if(df.values[i][0] < time_list[j]):
            tmp = j
            break
        
    if(tmp % 3 == 2 and df.values[i][2] == True):
        np_true_cnt += 1
    
    idx = tmp - 1

In [14]:
# np_true_cnt = False Alarm 감지 횟수
# 값이 클수록 모델의 성능 오류가 높다는 뜻
print(np_true_cnt)

0
